<a href="https://colab.research.google.com/github/OhJin-Soo/fintuned-model/blob/main/Kobert_fintuned_restaurant_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [5]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments
from sklearn.metrics import f1_score

In [6]:
# 1️⃣ TSV 파일 로드
df = pd.read_csv("/content/kr3.tsv", sep="\t")  # 컬럼: Review, Rating

# 2️⃣ label=2 제거 (모호한 리뷰)
df = df[df["Rating"] != 2].reset_index(drop=True)

# 3️⃣ 텍스트와 라벨 분리
texts = df["Review"].tolist()
labels = df["Rating"].tolist()  # 이제 0, 1만 남음

In [7]:
# 2️⃣ train/validation/test 분리
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    texts,
    labels,
    test_size=0.3,
    random_state=42,
    stratify=labels
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts,
    temp_labels,
    test_size=0.5,
    random_state=42,
    stratify=temp_labels
)

In [8]:
import numpy as np
print(np.unique(train_labels))
print(np.unique(val_labels))
print(np.unique(test_labels))

[0 1]
[0 1]
[0 1]


In [9]:
# 3️⃣ 모델 & 토크나이저
model_name = "skt/kobert-base-v1"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # num_labels=2

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
MAX_LEN = 128  # 또는 256, 절대 512 초과 X

train_encodings = tokenizer(
    train_texts,
    truncation=True,
    padding=True,
    max_length=MAX_LEN,
)

val_encodings = tokenizer(
    val_texts,
    truncation=True,
    padding=True,
    max_length=MAX_LEN,
)

test_encodings = tokenizer(
    test_texts,
    truncation=True,
    padding=True,
    max_length=MAX_LEN,
)

In [11]:
max_id = max([max(ids) for ids in train_encodings['input_ids']])
print(max_id, tokenizer.vocab_size)

8001 8002


In [12]:
# 5️⃣ PyTorch Dataset 생성
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {}

        item["input_ids"] = torch.tensor(
            self.encodings["input_ids"][idx], dtype=torch.long
        )
        item["attention_mask"] = torch.tensor(
            self.encodings["attention_mask"][idx], dtype=torch.long
        )

        # 🔥 핵심: token_type_ids 강제 0
        item["token_type_ids"] = torch.zeros_like(item["input_ids"])

        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item


train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

In [13]:
# 6️⃣ 데이터 길이 맞춤
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:
# 7️⃣ TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="kobert-sentiment-analysis-restaurant",
    learning_rate=5e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
)

In [15]:
# 8️⃣ Metrics 정의
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = np.mean(predictions == labels)
    f1 = f1_score(labels, predictions, average="macro")
    return {"accuracy": acc, "f1_macro": f1}

In [16]:
# 9️⃣ Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [17]:
model.config.max_position_embeddings

512

In [18]:
max_len_in_data = max(len(ids) for ids in train_encodings["input_ids"])
print("max sequence length:", max_len_in_data)
print("model max_position_embeddings:", model.config.max_position_embeddings)

max sequence length: 128
model max_position_embeddings: 512


In [19]:
sample = train_dataset[0]
print(sample["token_type_ids"].unique())

tensor([0])


In [20]:
# 10️⃣ 학습 시작
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: jin-soo (jin-soo-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.114200,0.093705,0.968517,0.920859
2,0.067500,0.096888,0.969681,0.924974
3,0.036000,0.110876,0.969332,0.924711


TrainOutput(global_step=3762, training_loss=0.07830845327341829, metrics={'train_runtime': 1379.7319, 'train_samples_per_second': 174.366, 'train_steps_per_second': 2.727, 'total_flos': 1.582474864686336e+16, 'train_loss': 0.07830845327341829, 'epoch': 3.0})

In [21]:
trainer.push_to_hub("Training complete!")

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...taurant/model.safetensors:   0%|          | 11.0kB /  369MB            

  ...s-restaurant/spiece.model: 100%|##########|  371kB /  371kB            

  ...48492.e22fe8a78b6c.1504.0:   2%|1         |   133B / 8.09kB            

  ...taurant/training_args.bin:   2%|1         |  97.0B / 5.91kB            

CommitInfo(commit_url='https://huggingface.co/jin-soo/kobert-sentiment-analysis-restaurant/commit/0385776eac5e9d20095174ee2bc99e8caa5a5074', commit_message='Training complete!', commit_description='', oid='0385776eac5e9d20095174ee2bc99e8caa5a5074', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jin-soo/kobert-sentiment-analysis-restaurant', endpoint='https://huggingface.co', repo_type='model', repo_id='jin-soo/kobert-sentiment-analysis-restaurant'), pr_revision=None, pr_num=None)

In [22]:
import transformers
transformers.logging.set_verbosity_error()

In [23]:
trainer.save_model("./best_model")  # best_model 폴더에 저장
tokenizer.save_pretrained("./best_model")  # 토크나이저도 저장

('./best_model/tokenizer_config.json',
 './best_model/special_tokens_map.json',
 './best_model/spiece.model',
 './best_model/added_tokens.json')

In [24]:
# 11️⃣ 테스트 평가
trainer.evaluate(test_dataset)

{'eval_loss': 0.09424202889204025,
 'eval_accuracy': 0.9699738143729997,
 'eval_f1_macro': 0.9257107670654469,
 'eval_runtime': 30.589,
 'eval_samples_per_second': 561.802,
 'eval_steps_per_second': 8.794,
 'epoch': 3.0}